In [1]:
!pip install pytorch_lightning
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.utils.data as data
import zipfile
import wandb


# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set the path to your zipped dataset
zip_file_path = '/content/drive/MyDrive/DL /aksharantar_sampled.zip'

# Extract the dataset to a folder in Google Drive
zip_ref = zipfile.ZipFile(zip_file_path, 'r')
zip_ref.extractall('aksharantar_sampled_extracted')
zip_ref.close()

In [4]:
import torch
import torch.nn as nn
from google.colab import drive
import csv
# from pytorch_lightning.loggers import WandbLogger
# move tensors to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# Path to your CSV file on Google Drive (Extracted file)
csv_file_path = '/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_train.csv'

# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []
    for row in reader:
        chars.extend(row[0])  # assuming the first column of the CSV file contains the text data

charS=set(chars)
charS.add('|')
char_set = list(charS)


# Define the mapping between characters and integer indices
char_to_idx_latin= {char: i+1 for i, char in enumerate(char_set)}


In [5]:
max_length_devanagari=0
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        chars.extend(row[1])  # assuming the first column of the CSV file contains the text data

# Define the character set
charS=set(chars)
charS.add('|')
char_set = list(charS)

# Define the mapping between characters and integer indices
char_to_idx_lang ={char: i+1 for i, char in enumerate(char_set)}

In [6]:

max_length_latin = 0

# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        length = len(row[0])  # assuming the column you want to check is the first one
        if length > max_length_latin:
            max_length_latin = length

In [7]:
# max_length_latin

In [8]:
max_length_devanagari = 0
# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        length = len(row[1])  # assuming the column you want to check is the first one
        if length > max_length_devanagari:
            max_length_devanagari = length

In [9]:
# char_to_idx_lang=char_to_idx_lang.to(device)
# char_to_idx_latin=char_to_idx_latin.to(device)

In [10]:
# char_set

In [11]:
# char_to_idx_latin

In [12]:
# max_length_devanagari

In [13]:
def word_to_indices(word, max_length,dict):
    # Convert the characters to integer indices using the char_to_idx mapping
    indices = [dict.get(c, -1) for c in word]
    # Filter out characters not in the dictionary
    indices = [idx for idx in indices if idx >= 0]
    # Add padding if necessary to make the sequence length equal to max_length
    if len(indices) < max_length:
        indices += [0] * (max_length - len(indices))
    # Truncate the sequence if necessary to make the sequence length equal to max_length
    elif len(indices) > max_length:
        indices = indices[:max_length]
    # Add the start and end tokens to the sequence
    indices = [dict['|']] + indices + [dict['|']]
    # Convert the list of indices to a tensor
    tensor = torch.tensor(indices)
    tensor=tensor.to(device)
    return tensor
pairs=[]

# Read the CSV file containing Latin-Devanagari word pairs
with open('/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_train.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        latin_word = row[0]
        devanagari_word = row[1]
        # Convert the Latin word to a tensor of integer indices
        latin_indices = word_to_indices(latin_word, max_length_latin,char_to_idx_latin)
        devanagari_indices= word_to_indices(devanagari_word,max_length_devanagari ,char_to_idx_lang)
        pairs.append([latin_indices,devanagari_indices])
        # print(devanagari_word, devanagari_indices)
        # print(latin_word,latin_indices)
        # print('**************************************************')

pairs_v=[]

# Read the CSV file containing Latin-Devanagari word pairs
with open('/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_valid.csv', 'r') as f_v:
    reader_v = csv.reader(f_v)
    for row in reader_v:
        latin_word = row[0]
        devanagari_word = row[1]
        # Convert the Latin word to a tensor of integer indices
        latin_indices = word_to_indices(latin_word, max_length_latin,char_to_idx_latin)
        devanagari_indices= word_to_indices(devanagari_word,max_length_devanagari ,char_to_idx_lang)
        pairs_v.append([latin_indices,devanagari_indices])
        # print(devanagari_word, devanagari_indices)
        # print(latin_word,latin_indices)
        # print('**************************************************')


In [14]:
# pairs_v

In [15]:
train_dataloader = torch.utils.data.DataLoader(pairs, batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(pairs_v, batch_size=64, shuffle=False)
# wandb_logger = WandbLogger(project="Assignment 3 Q1 a")
# a2e6402ce9fe2ebe1f01d5332c4fafa210b0dc0c


In [16]:
wandb.login(key="a2e6402ce9fe2ebe1f01d5332c4fafa210b0dc0c")
pName = "Assignment 3 sweep s"
run_obj=wandb.init( project=pName)

# Set up the configuration for the sweep using the `wandb.sweep` function
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        
        'drop_out': {"values": [0,0.2, 0.3]},

        'input_embedding_size': {"values": [16,32,64,256]},

        'hidden_layer_size': {"values": [16,32,64,256]},


        'number_of_encoder_layers': {"values": [1, 2, 3]},

        'number_of_decoder_layers': {"values": [1, 2, 3]},


        "cell_type": {
              "values": [ "RNN", "GRU", "LSTM"]
          },
          
          "learning_rate": {
              "values": [1e-3, 1e-4]
          },
       
        "bidirectional":{
            "values":[True, False]
        },

          "epochs": {
              "values": [10, 15, 20]
          },
    }
}

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs22m008. Use `wandb login --relogin` to force relogin


In [103]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embedding_size,cell_type,drop_out,num_layers,bidirectional):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding(input_dim, embedding_size)

        if(cell_type=="GRU"):
          self.rnn = nn.GRU(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers,bidirectional=bidirectional)
        if(cell_type=="LSTM"):
          self.rnn = nn.LSTM(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers,bidirectional=bidirectional)
        if(cell_type=="RNN"): 
          self.rnn = nn.RNN(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers,bidirectional=bidirectional)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        return output, hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim,embedding_size ,cell_type,drop_out,num_layers,bidirectional,max_length):
        self.max_length=max_length+2
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(output_dim, embedding_size)
        self.attn = nn.Linear(self.hidden_dim + embedding_size, self.max_length)
        self.bidirectional = bidirectional
        self.cell_type=cell_type
        D=1
        if self.bidirectional:
          D=2
        self.attn_combine = nn.Linear(self.hidden_dim*D + embedding_size, self.hidden_dim)

        if(cell_type=="GRU"):
          self.rnn = nn.GRU(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers ,bidirectional=bidirectional)
        elif(cell_type=="LSTM"):
          self.rnn = nn.LSTM(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers,bidirectional=bidirectional)
        elif(cell_type=="RNN"):
          self.rnn = nn.RNN(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers,bidirectional=bidirectional)

        self.fc = nn.Linear(hidden_dim*D, output_dim)

    def forward(self, x, hidden, encoder_outputs):

        x = x.unsqueeze(1).transpose(0,1)
        embedded = self.embedding(x)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs.permute(1,0,2))

        output = torch.cat((embedded[0], attn_applied.squeeze(1)), 1)

        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)

        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc(output.squeeze(0))

        
        return prediction, hidden, attn_weights

import torch.nn.functional as F

class Seq2Seq(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_dim,embedding_size, cell_type, drop_out,num_layers_encoders,num_layers_decoders,bidirectional,learning_rate,max_length_latin):
        super().__init__()

        self.val_loss=[]
        self.train_loss=[]

        self.train_accuracy=[]
        self.val_accuracy=[]
        self.encoder = Encoder(input_dim, hidden_dim, embedding_size,cell_type,drop_out,num_layers_encoders,bidirectional)
        self.decoder = Decoder(output_dim, hidden_dim, embedding_size, cell_type,drop_out,num_layers_decoders,bidirectional,max_length_latin)

        self.num_layers_encoders=num_layers_encoders
        self.num_layers_decoders=num_layers_decoders

        self.learning_rate=learning_rate
        self.cell_type=cell_type

        self.bidirectional = bidirectional
        self.D=1
        if self.bidirectional:
          self.D=2

    def forward(self, src, trg,teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.fc.out_features
        src = src.transpose(0,1)
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        encoder_output,hidden = self.encoder(src)

        input = trg[:,0] #start character
        for t in range(1, max_len):
            output, hidden,attn_weights = self.decoder(input, hidden,encoder_output)
            outputs[t] = output
            top1 = output.argmax(1)
            input = top1 if teacher_forcing_ratio < torch.rand(1).item() else trg[:,t]
            #if teacher_forcing_ratio > the random number generated then use the predicted character at the timestep t in timestamp t+1, else use the true value from the target 
        return outputs

    def training_step(self, batch, batch_idx):
        src, trg = batch
        # print("Before forward trg",trg.shape)
        output = self(src, trg)
        outputAcc = self(src, trg)
        trgAcc=trg 

        outputAcc = outputAcc.permute(1, 0, 2)
    
        cols = torch.arange(output.shape[1]).unsqueeze(1)        
        rows = torch.arange(output.shape[0])
        expected = torch.zeros(size=output.shape)
        expected[rows, cols, trg.cpu()] = 1

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        expected = expected[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        train_loss = self.loss_fn(output.to(device), expected.to(device))
        # print("loss ",loss)
        # self.log("loss", loss, prog_bar=True,on_epoch=True)

        train_accuracy =self.accuracy(outputAcc, trgAcc)    #trg is the true value
        # self.log("train_accuracy", train_accuracy, prog_bar=True,on_epoch=True)
        self.train_accuracy.append(torch.tensor(train_accuracy))
        self.train_loss.append(torch.tensor(train_loss))

        return {'loss': train_loss}

    def validation_step(self, batch, batch_idx):

        src, trg = batch
        output = self(src, trg,0)
        outputAcc = self(src, trg,0)
        trgAcc=trg

        outputAcc = outputAcc.permute(1, 0, 2)

        cols = torch.arange(output.shape[1]).unsqueeze(1)
        rows = torch.arange(output.shape[0])
        expected = torch.zeros(size=output.shape)
        expected[rows, cols, trg.cpu()] = 1
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        expected = expected[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)


        val_loss = self.loss_fn(output.to(device), expected.to(device))

        val_accuracy =self.accuracy(outputAcc, trgAcc)    #trg is the true value
        # self.log("val_accuracy", val_accuracy, prog_bar=True,on_epoch=True)
        self.val_accuracy.append(torch.tensor(val_accuracy))
        self.val_loss.append(torch.tensor(val_loss))

        return {'loss': val_loss}


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate )
        return optimizer

    def loss_fn(self, output, trg):
        # mask = (trg != self.pad_idx).float()
        criterion = nn.CrossEntropyLoss()
        # print("Loss cal",output.shape,trg.shape)
        loss = criterion(output, trg)
        # loss = criterion(output, trg) * mask
        return loss.mean()

    def accuracy(self, output, target):
      
      predicted = output.argmax(dim=-1)  # shape: (sequence_length, batch_size)
      equal_rows = 0
      for i in range(target.size(0)):
          # Remove the first and last indices and check if the sliced tensors are equal
          if torch.all(torch.eq(target[i, 1:-1], predicted[i, 1:-1])):
              equal_rows += 1
      matches=equal_rows

      # Compute the accuracy
      accuracy = matches / len(target) * 100
      return accuracy

    def on_train_epoch_end(self):

      train_loss=torch.stack(self.train_loss).mean()
      self.train_loss=[]

      val_loss=torch.stack(self.val_loss).mean()
      self.val_loss=[]

      train_accuracy=torch.stack(self.train_accuracy).mean()
      self.train_accuracy=[]

      val_accuracy=torch.stack(self.val_accuracy).mean()
      self.val_accuracy=[]

      wandb.log({"train_loss":train_loss,"train_accuracy":train_accuracy,"val_loss":val_loss,"val_accuracy":val_accuracy})
      print("train loss: ",train_loss,"    train accuracy:",train_accuracy,"   val loss: ",val_loss,"    val accuracy: ",val_accuracy)

def beam_search(self, encoder_output, encoder_hidden, decoder_input, beam_width,batch_size,max_len,trg_vocab_size,output):
    # Set initial beam
    beam = [(decoder_input, 0, encoder_hidden)]
    outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)


    # Iterate through the sequence
    for i in range(self.max_seq_len):
        new_beam = []
        for j in range(len(beam)):
            decoder_input = beam[j][0]
            decoder_hidden = beam[j][2]

            # Run the decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_output)

            # Get the top-k most probable tokens
            topk_probs, topk_indices = torch.topk(decoder_output, beam_width)

            for k in range(beam_width):
                # Calculate the new score
                score = beam[j][1] + topk_probs[0][k]

                # Create a new hypothesis
                hypothesis = (
                    torch.tensor([topk_indices[0][k]]).to(self.device),
                    score,
                    decoder_hidden
                )

                # Add it to the new beam
                new_beam.append(hypothesis)
        outputs[t] = output

        # Select the top-k hypotheses
        beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_width]

    return 

In [104]:
hidden_layer_size=256
embedding_size= 64
cell_type="GRU"
drop_out=0.2
num_layers_encoders=1
num_layers_decoders=1
bidirectional=True
learning_rate=0.001
epochs=20
model = Seq2Seq(len(char_to_idx_latin)+2, len(char_to_idx_lang)+2, hidden_layer_size, embedding_size, cell_type,drop_out,num_layers_encoders,num_layers_decoders,bidirectional,learning_rate,max_length_latin)
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(29, 64)
    (rnn): GRU(64, 256, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(67, 64)
    (attn): Linear(in_features=320, out_features=26, bias=True)
    (attn_combine): Linear(in_features=576, out_features=256, bias=True)
    (rnn): GRU(64, 256, dropout=0.2, bidirectional=True)
    (fc): Linear(in_features=512, out_features=67, bias=True)
  )
)

In [105]:
# def sweep():
#     wandb.init()
#     config = wandb.config


#     # print(config.activation_function,config.batch_normalization,config.data_augmentation,config.filter_organisation, config.drop_out)
    
#     # obj = CNNModel(config.activation_function,config.batch_normalization,config.data_augmentation,config.filter_organisation, config.drop_out)
#     model = Seq2Seq(len(char_to_idx_latin)+2, len(char_to_idx_lang)+2, config.hidden_layer_size, config.input_embedding_size, config.cell_type,config.drop_out,config.number_of_encoder_layers,config.number_of_decoder_layers,config.bidirectional,config.learning_rate)
#     model.to(device)
#     # trainer = pl.Trainer(max_epochs=config.epochs, accelerator="gpu", devices=1)
#     trainer = pl.Trainer(max_epochs=config.epochs)

#     trainer.fit(model=model,train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)

#     run_name = "lr_{}_hls_{}_es_{}_ct_{}_nle_{}_do_{}_nld_{}_bd_{}".format(config.learning_rate, config.hidden_layer_size,config.input_embedding_size,config.cell_type,config.number_of_encoder_layers, config.drop_out, config.number_of_decoder_layers,config.bidirectional)
#     print(run_name)
#     wandb.run.name = run_name
#     wandb.run.save()

# # Initialize the WandB sweep
# sweep_id = wandb.sweep(sweep_config,project=pName)
# wandb.agent(sweep_id, sweep)

In [106]:
# hidden_layer_size=256
# embedding_size= 64
# cell_type="GRU"
# drop_out=0.2
# num_layers_encoders=1
# num_layers_decoders=1
# bidirectional=True
# learning_rate=0.001
# epochs=20
# model = Seq2Seq(len(char_to_idx_latin)+2, len(char_to_idx_lang)+2, hidden_layer_size, embedding_size, cell_type,drop_out,num_layers_encoders,num_layers_decoders,bidirectional,learning_rate)
# model.to(device)

In [107]:
trainer = pl.Trainer(max_epochs=epochs, accelerator="gpu", devices=1)
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 496 K 
1 | decoder | Decoder | 689 K 
------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.743     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

<ipython-input-103-47bdff889285>:167: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.val_loss.append(torch.tensor(val_loss))


Training: 0it [00:00, ?it/s]

<ipython-input-103-47bdff889285>:139: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.train_loss.append(torch.tensor(train_loss))


...

Validation: 0it [00:00, ?it/s]

train loss:  tensor(0.1344, device='cuda:0')     train accuracy: tensor(61.9883)    val loss:  tensor(0.5086, device='cuda:0')     val accuracy:  tensor(30.7129)


Validation: 0it [00:00, ?it/s]

train loss:  tensor(0.1237, device='cuda:0')     train accuracy: tensor(63.7734)    val loss:  tensor(0.5371, device='cuda:0')     val accuracy:  tensor(32.6172)


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
